In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [13]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)

### 1-1. 기본 체인 만들기 - AI 요리사

In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    """
    너는 요리사야.
    {ingredients}를 가지고 만들 수 있는 요리 하나의 이름과 레시피 하나를 추천해줘.
    """)
prompt

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

try:
    result = chain2.invoke({"ingredients": "양파, 토마토, 치즈"})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

피자 조리법:

재료:

*   1.  밀가루 2컵 
*   2.  물 1컵 
*   3.  소금 1/2티스푼 
*   4.  설탕 1티스푼 
*   5.  이스트 1티스푼 
*   6.  올리브 오일 1티스푼 
*   7.  토마토소스 
*   8.  모짜렐라 치즈 
*   9.  다양한 토핑 (예: 소시지, 양파, 피망 등)

조리 방법:

1.  큰 그릇에 밀가루, 소금, 설탕, 이스트를 섞습니다.
2.  물과 올리브 오일을 넣고 반죽을 만들어줍니다.
3.  반죽을 10분 정도 손으로 치대줍니다.
4.  반죽을 실온에서 1시간 정도 발효시켜줍니다.
5.  반죽을 피자 모양으로 만들어줍니다.
6.  토마토소스를 바르고 모짜렐라 치즈를 뿌려줍니다.
7.  원하는 토핑을 추가해줍니다.
8.  오븐에 200도로 예열한 뒤 피자를 넣고 15분 정도 구워줍니다.


### 1-2. 2단계 체인 만들기 - 영화 추천 시스템

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가  입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3줄로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "action"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

**영화 제목: 존 윅 (2014)**

**줄거리 요약:**

*   전직 암살자 존 윅은 은퇴 후 평화로운 삶을 살고 있습니다.
*   그러나 옛 동료의 죽음으로 인해 다시 암살의 세계로 돌아오게 됩니다.
*   존 윅은 화려한 액션과 스토리 속에서 옛 동료의 죽음에 대한 복수를 위해 싸웁니다.


### 1-3. FewShotPromptTemplate과 시스템 메시지 활용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "news": "삼성전자가 AI 가속기를 출시할 예정이다.",
        "keywords": "삼성전자, AI가속기, 출시"
    },
    {
        "news": "WHO가 국제 협력의 중요성을 강조했다.",
        "keywords": "WHO, 국제협력, 보건"
    },
    {   
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행g
result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

키워드: 제미나이, 구글, AI
